In [1]:
import os
import pandas as pd
import numpy as np
import openpyxl

In [2]:
def combiner_type_1(nr, data_dir = '/home/dad/projects/tgu/test_task/data/'):
    ''' Принимает на вход номера файлов для обработки и директорию где эти файлы находятся,
        записывает результрующий файл в директорию по умолчанию и возвращает этот файл как pd.DataFrame.
        Обрабатывает только указанные файлы:
        обработка файлов №№ 3, 7, 8, 10, 23, 24, 26, 27, 31, 34, 35, 39, 44, 56, 58,
        индикаторов №№ 58683, 58684, 59597, 58672, 58525, 58484, 59557, 58527, 58712, 58681, 58702, 58807, 58700, 58517, 58468,
    '''
    result = pd.DataFrame(columns=('Расшифровка', 'Обозреваемый период', 'Единица измерения', 'Единица физических величин', 'Значение', 
                                   'Дата обновления данных', 'Пол', 'Возраст', 'Тип населенного пункта', 'Частота предоставления данных', 
                                   'Регион', 'Номер файла', 'Раздел'))
    for file_nr in nr:
        #  загружаем файл:
        data_name = data_dir + 'data(' + str(file_nr) + ').xls'
        dict_df = pd.read_excel(data_name, sheet_name=None)
        #  выделим в отдельные датафреймы Данные и Паспорт: 
        df = dict_df['Данные']
        df1 = dict_df['Паспорт']
        #  выделим значения из таблицы Данные:
        a1 = df.columns[0]
        a2 = df.iloc[2:, 0].astype(int)
        a3 = df.iloc[2:, 1]
        #  выставим значение для поля 'Единица физических величин':
        if file_nr not in (35, 39, 56):
            a4 = [df1.iloc[1, 1][2:]]
        elif file_nr == 35:
            a4 =['человек на миллион жителей']
        elif file_nr == 39:
            a4 = ['Количество исследователей (в эквиваленте полной занятости) на миллион жителей']
        elif file_nr == 56:
            a4 =['зарегистрированных больных на 100000 человек']
        length = len(a2) # длина получаемой таблицы
        #  сформируем таблицу по шаблону:
        output = pd.DataFrame({'Расшифровка': [a1] * length,
                          'Обозреваемый период': a2 if file_nr != 10 else [df.iloc[1, 1] + ', '] * 2 + df.iloc[2:, 0].astype(int).astype(str),
                          'Единица измерения': [df1.iloc[1, 1][2:]] * length,
                          'Единица физических величин': a4 * length, 
                          'Значение': df.iloc[2:, 1],
                          'Дата обновления данных': [df1.iloc[5, 1]] * length,
                          'Пол': ['_T'] * length,
                          'Возраст': ['_T'] * length,
                          'Тип населенного пункта': ['_T'] * length,
                          'Частота предоставления данных': ['A' if df1.iloc[2, 1][:9] == '- Годовая' else df1.iloc[2, 1][2:16]] * length,
                          'Регион': ['Российская Федерация'] * length,
                          'Номер файла': [file_nr] * length,
                          'Раздел': ['_T'] * length
                          })
        result = pd.concat([result, output], axis=0) #  добавляем сформированную таблицу в результирующую таблицу
    #  сохраним результаты в файл csv
    result.to_csv('result_type_1.csv', encoding='utf-8', index = False)
    return result

In [3]:
def combiner_type_8(nr, data_dir = '/home/dad/projects/tgu/test_task/data/'):
    ''' Принимает на вход номера файлов для обработки и директорию где эти файлы находятся,
        записывает результрующий файл в директорию по умолчанию и возвращает этот файл как pd.DataFrame.
        Обрабатывает только указанные файлы:
        обработка файлов №№ 0, 32, 33
        индикаторов №№ 58536, 59141, 40552
    '''
    result = pd.DataFrame(columns=('Расшифровка', 'Обозреваемый период', 'Единица измерения', 'Единица физических величин', 'Значение', 
                                   'Дата обновления данных', 'Пол', 'Возраст', 'Тип населенного пункта', 'Частота предоставления данных', 
                                   'Регион', 'Номер файла', 'Раздел'))
    for file_nr in nr:
        #  загружаем файл:
        data_name = data_dir + 'data(' + str(file_nr) + ').xls'
        dict_df = pd.read_excel(data_name, sheet_name=None)
        #  выделим в отдельные датафреймы Данные и Паспорт: 
        df = dict_df['Данные']
        df1 = dict_df['Паспорт']
        #  плавим исходную таблицу:
        if file_nr in (0, 33):
            df.iloc[1, 0:2] = (1, 2)
            df2 = df.rename(columns = df.iloc[1,:].astype(int))
            formatted_df = pd.melt(df2[2:], [1, 2], var_name="period", value_name="numbers")
        elif file_nr == 32:
            df.iloc[1, 0] = 1
            df2 = df.rename(columns = df.iloc[1,:].astype(int))
            formatted_df = pd.melt(df2[2:], [1], var_name="period", value_name="numbers")
        #  заменяем незначащие значения на _Т:
        mask = formatted_df[1].apply(lambda x: x == 'Всего%')
        formatted_df[1][mask] = '_T'
        #  выделим значения из таблицы Данные:
        a1 = df.columns[0] #  Расшифровка

        #  выставим значение для поля 'Единица физических величин':
        a4 = [df1.iloc[1, 1][2:]]

        length = len(formatted_df['period']) # длина получаемой таблицы
        #  сформируем таблицу по шаблону:
        output = pd.DataFrame({'Расшифровка': [a1] * length,
                          'Обозреваемый период': formatted_df['period'],
                          'Единица измерения': [df1.iloc[1, 1][2:]] * length,
                          'Единица физических величин': a4 * length, 
                          'Значение': formatted_df['numbers'],
                          'Дата обновления данных': [df1.iloc[5, 1]] * length,
                          'Пол': ['_T'] * length,
                          'Возраст': ['_T'] * length,
                          'Тип населенного пункта': ['_T'] * length,
                          'Частота предоставления данных': ['A' if df1.iloc[2, 1][:9] == '- Годовая' else df1.iloc[2, 1][2:16]] * length,
                          'Регион': formatted_df[2].apply(lambda x: x.lstrip()) if file_nr == 0 else ['Российская Федерация'] * length,
                          'Номер файла': [file_nr] * length,
                          'Раздел': formatted_df[1].apply(lambda x: x.lstrip()),
                          })

        result = pd.concat([result, output], axis=0) #  добавляем сформированную таблицу в результирующую таблицу
    #  сохраним результаты в файл csv
    result.to_csv('result_type_8.csv', encoding='utf-8', index = False)
    return result

In [4]:
c1 = combiner_type_1([3, 7, 8, 10, 23, 24, 26, 27, 31, 34, 35, 39, 44, 56, 58])
c8 = combiner_type_8([0, 32, 33])

/tmp/ipykernel_26690/1395900439.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  formatted_df[1][mask] = '_T'


In [5]:
c = pd.concat([c1, c8], axis=0) #  объединим промежуточные результаты
filtered = c[~c['Значение'].isna() & (c['Значение'] != 0)] #  отфильтруем ноли и пропуски в значениях
filtered.to_csv('tidy_data.csv', encoding='utf-8', index = False) #  выгрузим конечный результат